In [1]:
#include "includeLibraries.h"

In [2]:
#include <string>
#include <cstring>
#include <fstream>
#include <dlib/opencv.h>
#include <dlib/image_processing.h>
#include <dlib/image_processing/frontal_face_detector.h>
#include <opencv2/opencv.hpp>

In [3]:
// Write landmarks to file
void writeLandmarksToFile(
    dlib::full_object_detection &landmarks, 
    const std::string &filename
) {
    // Open file
    std::ofstream ofs;
    ofs.open(filename);
  
    // Loop over all landmark points
    for (int i = 0; i < landmarks.num_parts(); i++) {
        // Print x and y coordinates to file
        ofs << landmarks.part(i).x() << " " << landmarks.part(i).y() << std::endl;
    }
    // Close file
    ofs.close();
}

In [4]:
void drawLandmarks(cv::Mat &im, dlib::full_object_detection landmarks) {
    for(int i = 0; i < landmarks.num_parts(); i++) {
        int px = landmarks.part(i).x();
        int py = landmarks.part(i).y();

        cv::circle(im, cv::Point(px, py), 1, cv::Scalar(0, 0, 255), 2, cv::LINE_AA);
        cv::putText(
            im, 
            std::to_string(i + 1), 
            cv::Point(px, py), 
            cv::FONT_HERSHEY_SIMPLEX, 
            .3, 
            cv::Scalar(255, 0, 0), 
            1
        );
    }
}

In [5]:
dlib::frontal_face_detector faceDetector = dlib::get_frontal_face_detector();

In [6]:
dlib::shape_predictor landmarkDetector;

In [7]:
dlib::deserialize("shape_predictor_68_face_landmarks.dat") >> landmarkDetector;

In [8]:
std::string imageFilename("girl.jpg");

In [9]:
cv::Mat im = cv::imread(imageFilename);

In [10]:
std::string landmarksBasename("output");

In [11]:
dlib::cv_image<dlib::bgr_pixel> dlibIm(im);

In [12]:
std::vector<dlib::rectangle> faceRects;

In [13]:
faceRects = faceDetector(dlibIm);

In [14]:
std::cout << "Number of faces detected: " << faceRects.size() << std::endl;

Number of faces detected: 1


In [15]:
std::vector<dlib::full_object_detection> landmarksAll;

In [16]:
for (int i = 0; i < faceRects.size(); i++) {
    // For every face rectangle, run landmarkDetector
    dlib::full_object_detection landmarks;
    
    landmarks = landmarkDetector(dlibIm, faceRects[i]);
    
    // Print number of landmarks
    std::cout << "Number of landmarks : " << landmarks.num_parts() << std::endl;
    
    // Store landmarks for current face
    landmarksAll.push_back(landmarks);
    
    // Draw landmarks on face
    drawLandmarks(im, landmarks);

    // Write landmarks to disk
    std::stringstream landmarksFilename;
    landmarksFilename << landmarksBasename <<  "_"  << i << ".txt";
    std::cout << "Saving landmarks to " << landmarksFilename.str() << std::endl;
    writeLandmarksToFile(landmarks, landmarksFilename.str());
}

Number of landmarks : 68
Saving landmarks to output_0.txt


In [17]:
std::string outputFilename("result_Landmarks.jpg");
std::cout << "Saving output image to " << outputFilename << std::endl;

Saving output image to result_Landmarks.jpg


In [18]:
cv::imwrite(outputFilename, im);